# Data Engineering and Analytics
## Assignment 1
### Data Extraction and Integration Challenge

# __________________________________________________________________

### Belal Khaled(2136873) - Yaseen Zaqlam(2130397)

# __________________________________________________________________

# • Importing all libraries we need:

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float

# __________________________________________________________________

# • Task 1: Data Scraping

### ○ Function to scrape data from Amazon search results page:

In [2]:
def scrape_amazon_page(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "DNT": "1",
    "Referer": "https://www.amazon.com/",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1"
}
    product_data = []  # List to store product data
    while True:  # Infinite loop until getting a 200 response
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser') # returns html doc
            # Extract data from the page
            products = soup.find_all('div', {'data-component-type': 's-search-result'})
            for product in products:
                # Extract image URL
                image_element = product.find('img', {'class': 's-image'})
                image_url = image_element['src'] if image_element else 'NaN'
                # Extract product title
                title_element = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
                title = title_element.text.strip() if title_element else 'NaN' #strip removes whitespaces(returns string)
                # Extract product link
                link_element = product.find('a', {'class': 'a-link-normal'})
                link = 'https://www.amazon.com' + link_element['href'] if link_element else 'NaN'
                # Extract product price
                price_element = product.find('span', {'class': 'a-offscreen'})
                price = price_element.text.strip() if price_element else 'NaN' #strip removes whitespaces(returns string)
                # Extract product rating
                rating_element = product.find('span', {'class': 'a-icon-alt'})
                rating = rating_element.text.strip().split()[0] if rating_element else 'NaN' #split returns it as a list
                # Extract number of user reviews
                num_reviews_element = product.find('span', {'class': 'a-size-base'})
                num_reviews = num_reviews_element.text.strip().split()[0] if num_reviews_element else 'NaN' #split returns it as a list
                # Extract availability status
                availability_element = product.find('span', {'class': 'a-size-base a-color-price'})
                availability = availability_element.text.strip() if availability_element else 'Available' #strip removes whitespaces(returns string)
                # Append data to the list
                product_data.append({'Image URL': image_url, 'Title': title, 'Link': link, 'Price': price, 
                                     'Rating': rating,'Number of Reviews': num_reviews, 'Availability': availability})
            break  # Exit the loop if successful
        else:
            print("Failed to fetch page:", response.status_code)
            # Introduce a delay before retrying
            time.sleep(random.randint(2, 5))
    return product_data

### ○ Function to scrape multiple pages:

In [3]:
def scrape_multiple_pages(base_url, num_pages):
    all_product_data = []  # List to store data from all pages
    for page_num in range(1, num_pages + 1):
        url = f"{base_url}&page={page_num}"
        print(f"Scraping page {page_num}: {url}")
        product_data = scrape_amazon_page(url)
        all_product_data.extend(product_data)
        # Introduce a delay to avoid overwhelming the server
        time.sleep(random.randint(2, 5))  
    return all_product_data

#### - URL of the Amazon search results page for laptops:
#### - Number of pages to scrape:

In [4]:
base_url = "https://www.amazon.com/s?k=laptops&crid=1H049JIFTK5QX&sprefix=laptop%2Caps%2C212&ref=nb_sb_noss_1"
num_pages = 20

### ○ Call the function to scrape multiple pages:

In [5]:
all_product_data = scrape_multiple_pages(base_url, num_pages)

Scraping page 1: https://www.amazon.com/s?k=laptops&crid=1H049JIFTK5QX&sprefix=laptop%2Caps%2C212&ref=nb_sb_noss_1&page=1
Failed to fetch page: 503
Failed to fetch page: 503
Scraping page 2: https://www.amazon.com/s?k=laptops&crid=1H049JIFTK5QX&sprefix=laptop%2Caps%2C212&ref=nb_sb_noss_1&page=2
Failed to fetch page: 503
Scraping page 3: https://www.amazon.com/s?k=laptops&crid=1H049JIFTK5QX&sprefix=laptop%2Caps%2C212&ref=nb_sb_noss_1&page=3
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Failed to fetch page: 503
Scraping page 4: https://www.amazon.com/s?k=laptops&crid=1H049JIFTK5QX&sprefix=laptop%2Caps%2C212&ref=nb_sb_noss_1&page=4
Failed to fetch page: 503
Failed to fetch pa

#### - Create a DataFrame from the scraped data:
#### - Save the DataFrame to an Excel file:


In [6]:
df = pd.DataFrame(all_product_data)
df.to_excel('amazon_laptops.xlsx', index=False)

#first 5 rows:
df.head()

,Image URL,Title,Link,Price,Rating,Number of Reviews,Availability
0,https://m.media-amazon.com/images/I/71pTP-ll4s...,"HP 15.6"" Portable Laptop (Include 1 Year Micro...",https://www.amazon.com/HP-Portable-Microsoft-Q...,$309.99,4.3,Amazon's,Available
1,https://m.media-amazon.com/images/I/61gKkYQn6l...,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,https://www.amazon.com/A315-24P-R7VH-Display-Q...,NaN,4.3,"1,747",Available
2,https://m.media-amazon.com/images/I/71CDSpds6j...,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",https://www.amazon.com/HP-Display-i3-1125G4-Pr...,NaN,4.4,572,Available
3,https://m.media-amazon.com/images/I/711OHeRmEa...,"HP Newest 14"" Ultral Light Laptop for Students...",https://www.amazon.com/HP-Students-Business-Qu...,NaN,4.2,"1,806",Available
4,https://m.media-amazon.com/images/I/815uX7wkOZ...,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",https://www.amazon.com/HP-Micro-edge-Microsoft...,$186.94,3.9,"1,291",Available


# __________________________________________________________________

# • Task 2: Data Preprocessing and Transformation

In [7]:
#Extracting DataFrame(df) info:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Image URL          425 non-null    object
 1   Title              425 non-null    object
 2   Link               425 non-null    object
 3   Price              425 non-null    object
 4   Rating             425 non-null    object
 5   Number of Reviews  425 non-null    object
 6   Availability       425 non-null    object
dtypes: object(7)
memory usage: 23.4+ KB


In [8]:
#Count the occurrences of 'NaN' in each column
NaN_counts = df.apply(lambda x: x.value_counts().get('NaN', 0))
null_counts = df.isnull().sum()

#Print the counts of NaN
print("Number of 'NaN' occurrences in each column:")
print(NaN_counts)

#Print the counts of null values
print("Number of null values:")
print(null_counts)

Number of 'NaN' occurrences in each column:
Image URL              0
Title                  0
Link                   0
Price                234
Rating                26
Number of Reviews     12
Availability           0
dtype: int64
Number of null values:
Image URL            0
Title                0
Link                 0
Price                0
Rating               0
Number of Reviews    0
Availability         0
dtype: int64


### ○ Takes a copy of DataFrame(df) to start Data Preprocessing and Transformation:

In [9]:
df2 = df.copy()

#Keys of df2 (test):
df2.keys()

Index(['Image URL', 'Title', 'Link', 'Price', 'Rating', 'Number of Reviews',
       'Availability'],
      dtype='object')

# __________________________________________________________________

## • Task 2.1
### • Data Cleaning:

### ○ Starting with 'Price' column:

In [10]:
df2['Price'] = df2['Price'].str.replace('$', '') #removing $ sign 
df2['Price'] = df2['Price'].str.replace(',', '') #removing ,
df2['Price'] = df2['Price'].astype(float) #changing the Price datatype to float

C:\Users\MIS\AppData\Local\Temp/ipykernel_16128/899882573.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Price'] = df2['Price'].str.replace('$', '') #removing $ sign


In [11]:
df2['Price'].head()

0    309.99
1       NaN
2       NaN
3       NaN
4    186.94
Name: Price, dtype: float64

### ○ Then 'Rating' cloumn:

In [12]:
df2['Rating'] = df2['Rating'].astype(float) #changing the Rating datatype to float

In [13]:
df2['Rating'].head()

0    4.3
1    4.3
2    4.4
3    4.2
4    3.9
Name: Rating, dtype: float64

### ○ Finally 'Number of Reviews' cloumn:

In [14]:
df2['Number of Reviews'].unique()

array(["Amazon's", '1,747', '572', '1,806', '1,291', '822', '40', '72',
       '943', '50', '309', '51', '208', '66', '580', '2,448', '2', '11',
       '63', '587', '58', '502', '68', '91', '573', '27', '274', '32',
       '458', '1,052', '1,874', '35', '467', '418', '202', '28', 'Only',
       'NaN', '167', '4,196', '93', '386', '67', '469', '4', '230', '6',
       '18', '252', '187', '81', '10', '80', '468', '85', '188', '119',
       '8', '2,542', '144', '9', '191', '3', '1,167', '135', '87', '354',
       '4,388', '88', '410', '15', '131', '3,369', '43', '31', '306',
       '89', '62', '47', '243', '13', '30', '2,486', '253', '177', '37',
       '165', '214', '48', '372', '20,112', '10,325', '26', '3,850', '25',
       '14', '557', '36', '73', '21', '611', '342', '290', '332', '1',
       '46', '498', '23', '42', '5,979', '415', '76', '183', '8,567',
       '426', '634', '146', '151', '125', '19', '100', '222', '228',
       '136', '1,323', '52', '82', '61', '409', '20', '876', '30

In [15]:
df2['Number of Reviews'] = df2['Number of Reviews'].str.replace(r"[A-z':,+]", '',regex=True)
df2['Number of Reviews'] = df2['Number of Reviews'].str.replace(r'[\s]+','NaN',regex=True)
df2['Number of Reviews'] = df2['Number of Reviews'].replace('', 'NaN')
df2['Number of Reviews'] = df2['Number of Reviews'].astype(float) #changing the Number of Reviews datatype to float

In [16]:
df2['Number of Reviews'].head()

0       NaN
1    1747.0
2     572.0
3    1806.0
4    1291.0
Name: Number of Reviews, dtype: float64

### ○ Changing the datatypes from float64 to float32 for these columns:

In [17]:
df2 = df2.astype({'Price': 'float32', 'Rating': 'float32','Number of Reviews':'float32'})

In [18]:
df2.info() #Checking

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Image URL          425 non-null    object 
 1   Title              425 non-null    object 
 2   Link               425 non-null    object 
 3   Price              191 non-null    float32
 4   Rating             399 non-null    float32
 5   Number of Reviews  396 non-null    float32
 6   Availability       425 non-null    object 
dtypes: float32(3), object(4)
memory usage: 18.4+ KB


### ○ Handeling Missing Values (NaN) BY MEAN:

In [19]:
#Extracting the mean values for these columns
Mean=[df2['Price'].mean(),df2['Rating'].mean(),df2['Number of Reviews'].mean()]
Mean

[495.02728271484375, 4.230831146240234, 343.43182373046875]

In [20]:
df2['Price'].fillna(df2['Price'].mean(), inplace=True)
df2['Rating'].fillna(df2['Rating'].mean(), inplace=True)
df2['Number of Reviews'].fillna(df2['Number of Reviews'].mean(), inplace=True)
df2

,Image URL,Title,Link,Price,Rating,Number of Reviews,Availability
0,https://m.media-amazon.com/images/I/71pTP-ll4s...,"HP 15.6"" Portable Laptop (Include 1 Year Micro...",https://www.amazon.com/HP-Portable-Microsoft-Q...,309.989990,4.3,343.431824,Available
1,https://m.media-amazon.com/images/I/61gKkYQn6l...,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,https://www.amazon.com/A315-24P-R7VH-Display-Q...,495.027283,4.3,1747.000000,Available
2,https://m.media-amazon.com/images/I/71CDSpds6j...,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",https://www.amazon.com/HP-Display-i3-1125G4-Pr...,495.027283,4.4,572.000000,Available
3,https://m.media-amazon.com/images/I/711OHeRmEa...,"HP Newest 14"" Ultral Light Laptop for Students...",https://www.amazon.com/HP-Students-Business-Qu...,495.027283,4.2,1806.000000,Available
4,https://m.media-amazon.com/images/I/815uX7wkOZ...,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",https://www.amazon.com/HP-Micro-edge-Microsoft...,186.940002,3.9,1291.000000,Available
...,...,...,...,...,...,...,...
420,https://m.media-amazon.com/images/I/717lfaecwB...,"HP Pavilion Plus 14 inch Laptop, 2.8K OLED Dis...",https://www.amazon.com/HP-Pavilion-Generation-...,495.027283,4.3,51.000000,Available
421,https://m.media-amazon.com/images/I/71SVP-CL5Y...,"LG gram 14-inch Lightweight Laptop, Intel Evo ...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027283,4.3,12.000000,Available
422,https://m.media-amazon.com/images/I/71a7M-Sd1X...,"LG gram 15-inch Lightweight Laptop, Intel Evo ...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027283,4.4,14.000000,Available
423,https://m.media-amazon.com/images/I/71fRXRI3V7...,"LG gram 14-inch 2in1 Lightweight Laptop, Intel...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027283,4.0,2.000000,Available


In [21]:
#checking for null values
df2.isnull().sum()

Image URL            0
Title                0
Link                 0
Price                0
Rating               0
Number of Reviews    0
Availability         0
dtype: int64

# __________________________________________________________________

## • Task 2.2 
### • Transform the Data:

In [22]:
#Task 1: Standardize Prices
scaler = MinMaxScaler()
df2['Standardized Price'] = scaler.fit_transform(df2[['Price']])

#Task 2: Encode the 'Availability' Column
encoder = LabelEncoder()
df2['Encoded Availability'] = encoder.fit_transform(df2['Availability'])

#Task 3: Extract Features from 'Title'
#Here we will extract a simple feature - checking for the presence of 'Touchscreen'
df2['Has Touchscreen'] = df2['Title'].apply(lambda x: 'Touchscreen' in x)

#saving the transformed dataset
df2.to_excel('transformed_data.xlsx')

df2.head()

,Image URL,Title,Link,Price,Rating,Number of Reviews,Availability,Standardized Price,Encoded Availability,Has Touchscreen
0,https://m.media-amazon.com/images/I/71pTP-ll4s...,"HP 15.6"" Portable Laptop (Include 1 Year Micro...",https://www.amazon.com/HP-Portable-Microsoft-Q...,309.989990,4.3,343.431824,Available,0.067546,0,False
1,https://m.media-amazon.com/images/I/61gKkYQn6l...,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,https://www.amazon.com/A315-24P-R7VH-Display-Q...,495.027283,4.3,1747.000000,Available,0.116368,0,False
2,https://m.media-amazon.com/images/I/71CDSpds6j...,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",https://www.amazon.com/HP-Display-i3-1125G4-Pr...,495.027283,4.4,572.000000,Available,0.116368,0,False
3,https://m.media-amazon.com/images/I/711OHeRmEa...,"HP Newest 14"" Ultral Light Laptop for Students...",https://www.amazon.com/HP-Students-Business-Qu...,495.027283,4.2,1806.000000,Available,0.116368,0,False
4,https://m.media-amazon.com/images/I/815uX7wkOZ...,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",https://www.amazon.com/HP-Micro-edge-Microsoft...,186.940002,3.9,1291.000000,Available,0.035079,0,False


# __________________________________________________________________

## • Task 2.3
### • Integration:

In [23]:
integrated_df = df2.copy()

#saving it to a csv format
integrated_df.to_csv('integrated_dataset.csv', index=False)
integrated_df.dtypes

Image URL                object
Title                    object
Link                     object
Price                   float32
Rating                  float32
Number of Reviews       float32
Availability             object
Standardized Price      float32
Encoded Availability      int32
Has Touchscreen            bool
dtype: object

#### ○ Preparing the data to load it to a Database successfully:

In [24]:
# Defining data types based on DataFrame columns to suit with Database
sql_dtypes = {
    'Image URL': Text(),
    'Title': Text(),
    'Link': Text(),
    'Price':Float(),
    'Rating':Float(),
    'Number of Reviews':Float(),
    'Availability':Text(),
    'Standardized Price':Float(),
    'Encoded Availability':Integer(),
    'Has Touchscreen':Integer()
}

# __________________________________________________________________

# • Task 3: Loading Data and Executing Queries in MySQL Database Server

### • Task 3.B: Connect to the MySQL Database

In [25]:
# Database Information
db_user = 'root'
db_password = 'bkshbm1234#_'
#db_password = 'Bkshbm1234#_'
db_host = 'localhost'
db_port = '3306'
db_name = 'mysql'

#MySQL connection string
connection_string = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

#Create the engine
engine = create_engine(connection_string)

#Test the connection
try:
    with engine.connect() as conn:
        print("Connected to MySQL database successfully!")
except Exception as e:
    print(f"Failed to connect to MySQL database: {e}")

Connected to MySQL database successfully!


### • Task 3.C: Create a Database using Python Code

In [26]:
#Create a new database
new_db_name = 'new_database'

In [27]:
try:
    with engine.connect() as conn:
        conn.execute(f"CREATE DATABASE {new_db_name};")
        print(f"Database '{new_db_name}' created successfully!")
except Exception as e:
    print(f"Failed to create database: {e}")

Failed to create database: (mysql.connector.errors.DatabaseError) 1007 (HY000): Can't create database 'new_database'; database exists
[SQL: CREATE DATABASE new_database;]
(Background on this error at: https://sqlalche.me/e/14/4xp6)


### • Task 3.D: Load the Integrated Dataset into the Database

In [28]:
#Load data into the database, explicitly using the database
try:
    with engine.connect() as conn:
        conn.execute(f"USE {new_db_name};")  # Ensure we are in the right database
        integrated_df.to_sql('new_database', con=engine, index=False, if_exists='replace', dtype=sql_dtypes)
        print("Data loaded into MySQL database successfully!")
except Exception as e:
    print(f"Failed to load data into MySQL database: {e}")

Data loaded into MySQL database successfully!


### • Task.E: Execute at Least 3 Queries of Your Choice
#### ○ Querie 1:

In [29]:
#Execute the SQL query to retrieve all column names
Querie1 = f"SHOW COLUMNS FROM {new_db_name};"
result1 = engine.execute(Querie1)

#Fetch and print the result
for column in result1:
    print(column[0])  

Image URL
Title
Link
Price
Rating
Number of Reviews
Availability
Standardized Price
Encoded Availability
Has Touchscreen


#### ○ Querie 2:

In [65]:
#Execute the SQL query to retrieve the top 10 records from the table
Query2 = f"SELECT * FROM {new_db_name} LIMIT 10;"
result2 = engine.execute(Query2)

#Fetch the result and print it
result2

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'new_database.new_database' doesn't exist
[SQL: SELECT * FROM new_database LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/14/f405)

#### ○ Querie 3:

In [64]:
#Execute the SQL query to calculate summary statistics
Query3 = engine.execute(f"SELECT MAX(Price) AS max_price FROM {new_db_name};")
Query3_2 = engine.execute(f"SELECT MAX('Standardized Price') AS max_price_2 FROM {new_db_name};")

#Fetch the maximum value from the result
max_price = Query3.fetchone()[0]
max_price_2 = Query3_2.fetchone()[0]

print("Maximum price:", max_price)
print("Maximum Standardized Price:", max_price_2)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'new_database.new_database' doesn't exist
[SQL: SELECT MAX(Price) AS max_price FROM new_database;]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [33]:
#Execute the SQL query to retrieve all content from the table
result = engine.execute(f"SELECT * FROM {new_db_name};")

#Fetch all rows from the result
rows = result.fetchall()

#Convert the result to a DataFrame
df45 = pd.DataFrame(rows, columns=result.keys())
df45

,Image URL,Title,Link,Price,Rating,Number of Reviews,Availability,Standardized Price,Encoded Availability,Has Touchscreen
0,https://m.media-amazon.com/images/I/71pTP-ll4s...,"HP 15.6"" Portable Laptop (Include 1 Year Micro...",https://www.amazon.com/HP-Portable-Microsoft-Q...,309.990,4.3,343.432,Available,0.067546,0,0
1,https://m.media-amazon.com/images/I/61gKkYQn6l...,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,https://www.amazon.com/A315-24P-R7VH-Display-Q...,495.027,4.3,1747.000,Available,0.116368,0,0
2,https://m.media-amazon.com/images/I/71CDSpds6j...,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",https://www.amazon.com/HP-Display-i3-1125G4-Pr...,495.027,4.4,572.000,Available,0.116368,0,0
3,https://m.media-amazon.com/images/I/711OHeRmEa...,"HP Newest 14"" Ultral Light Laptop for Students...",https://www.amazon.com/HP-Students-Business-Qu...,495.027,4.2,1806.000,Available,0.116368,0,0
4,https://m.media-amazon.com/images/I/815uX7wkOZ...,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",https://www.amazon.com/HP-Micro-edge-Microsoft...,186.940,3.9,1291.000,Available,0.035079,0,0
...,...,...,...,...,...,...,...,...,...,...
420,https://m.media-amazon.com/images/I/717lfaecwB...,"HP Pavilion Plus 14 inch Laptop, 2.8K OLED Dis...",https://www.amazon.com/HP-Pavilion-Generation-...,495.027,4.3,51.000,Available,0.116368,0,0
421,https://m.media-amazon.com/images/I/71SVP-CL5Y...,"LG gram 14-inch Lightweight Laptop, Intel Evo ...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027,4.3,12.000,Available,0.116368,0,0
422,https://m.media-amazon.com/images/I/71a7M-Sd1X...,"LG gram 15-inch Lightweight Laptop, Intel Evo ...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027,4.4,14.000,Available,0.116368,0,0
423,https://m.media-amazon.com/images/I/71fRXRI3V7...,"LG gram 14-inch 2in1 Lightweight Laptop, Intel...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,495.027,4.0,2.000,Available,0.116368,0,0


# THE END.

# __________________________________________________________________